## Convolutional NN to classify govuk content to level2 taxons

Based on:
https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

## To do:
- ~~Consider grouping very small classes (especially if too small for evaluation)~~
- ~~Split data into training, validation and test to avoid overfitting validation data during hyperparamter searches & model architecture changes~~
- ~~Try learning embeddings~~--
- ~~Try changing pos_ratio~~
- Try implementing class_weights during model fit (does this do the same as the weighted binary corss entropy?)
- Work on tensorboard callbacks
- ~~Create dictionary of class indices to taxon names for viewing results~~
- ~~Check model architecture~~
- ~~consider relationship of training error to validation error - overfitting/bias?~~
- ~~train longer~~
- Try differnet max_sequence_length
- Check batch size is appropriate
- Also think about:
  - ~~regularization (e.g. dropout)~~ 
  - fine-tuning the Embedding layer

### Load requirements and data

TODO: edit requirement.txt to include only these packages and do not include tensorflow because this conflicts with tf on AWS when using on GPU.

In [34]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.utils import to_categorical, layer_utils, plot_model

from keras.layers import (Embedding, Input, Dense, Dropout, 
                          Activation, Conv1D, MaxPooling1D, Flatten, concatenate, Reshape)
from keras.models import Model, Sequential
from keras.optimizers import rmsprop
from keras.callbacks import TensorBoard, Callback, ModelCheckpoint
import keras.backend as K
from keras.losses import binary_crossentropy

from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score 
from sklearn.metrics import precision_recall_fscore_support, classification_report
from sklearn.utils import class_weight

import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

import functools

import h5py


### Environmental vars

In [35]:
DATADIR=os.getenv('DATADIR')
#DATADIR='/data' #this was put in for AWS run but doesn't work locally...

## Hyperparameters

Intuition for POS_RATIO is that it penalises the prediction of zero for everything, which is attractive to the model because the multilabel y matrix is super sparse. 

Increasing POS_RATIO should penalise predicting zeros more.

In [36]:
#MAX_NB_WORDS
MAX_SEQUENCE_LENGTH =1000
EMBEDDING_DIM = 100 # keras embedding layer output_dim = Dimension of the dense embedding
P_THRESHOLD = 0.5 #Threshold for probability of being assigned to class
POS_RATIO = 0.5 #ratio of positive to negative for each class in weighted binary cross entropy loss function
NUM_WORDS=20000 #keras tokenizer num_words: None or int. Maximum number of words to work with 
#(if set, tokenization will be restricted to the top num_words most common words in the dataset).

### Read in data
Content items tagged to level 2 taxons or lower in the topic taxonomy

In [92]:
labelled_level2 = pd.read_csv(os.path.join(DATADIR, 'labelled_level2.csv.gz'), dtype=object, compression='gzip')

In [93]:
labelled_level2.shape

(173560, 23)

In [94]:
labelled_level2['content_id'].nunique()

114048

### DON'T Collapse taxons with insufficient support for predictions (will need to be manually tagged)

In [95]:
#count the number of taxons per content item into new column
#labelled_level2['num_content_per_taxon'] = labelled_level2.groupby(["level2taxon"])['level2taxon'].transform("count")

In [96]:
#COLLAPSE level2taxons with too few content items into "toosmall" category
#labelled_level2.loc[labelled_level2['num_content_per_taxon'] < 10, 'level2taxon'] = 'TOO_SMALL'

#### clean up any World taxons leftover despite dropping relevant doctypes

In [97]:
#COLLAPSE World level2taxons
labelled_level2.loc[labelled_level2['level1taxon'] == 'World', 'level2taxon'] = 'world_level1'

#creating categorical variable for level2taxons from values
labelled_level2['level2taxon'] = labelled_level2['level2taxon'].astype('category')

### Upsample minority classes to address imbalance leading to ~2, 465, 570 rows of data!

In [98]:
#count the number of content items per taxon into new column
labelled_level2['num_content_per_taxon'] = labelled_level2.groupby(["level2taxon"])['level2taxon'].transform("count")

In [99]:
labelled_level2['num_content_per_taxon'].describe()

count    173560.000000
mean       4574.207145
std        3682.635048
min           1.000000
25%        1500.000000
50%        3780.000000
75%        6156.000000
max       11717.000000
Name: num_content_per_taxon, dtype: float64

In [100]:
#number of rows in biggest level2 taxon -this is the target size for all other level2 taxons in resampling
max_content_freq = max(labelled_level2['num_content_per_taxon'])
max_content_freq

11717

In [116]:
from sklearn.utils import resample
balanced_df = labelled_level2
for taxon in range(0, labelled_level2['level2taxon'].nunique()):
    print(taxon)
    # Separate majority and minority classes
    df_minority = labelled_level2[labelled_level2.level2taxon.cat.codes==taxon]
 
    # Upsample minority class
    df_minority_upsampled = resample(df_minority, 
                                         replace=True,     # sample with replacement
                                         n_samples=(max_content_freq-len(labelled_level2[labelled_level2['level2taxon'].cat.codes==taxon])),    # to match majority class, switch to max_content_freq if works
                                         random_state=123) # reproducible results
    
    # Combine majority class with upsampled minority class
    balanced_df = pd.concat([balanced_df, df_minority_upsampled])
 
    # Display new class counts
    print(balanced_df.level2taxon.value_counts())


0
Administrative justice reform                                   11717
Business and enterprise                                         11717
Government efficiency, transparency and accountability          11049
UK economy                                                      10527
Trade and investment                                             7229
Public health                                                    6156
National security                                                5765
Climate change and energy                                        5675
Foreign affairs                                                  5010
National Health Service                                          4970
Community and society                                            4334
Housing                                                          3996
Wildlife and animal welfare                                      3944
Housing planning and building                                    3826
International aid 

Administrative justice reform                             11717
Adoption, fostering and surrogacy                         11717
Afghanistan                                               11717
Business and enterprise                                   11717
Government efficiency, transparency and accountability    11049
UK economy                                                10527
Trade and investment                                       7229
Public health                                              6156
National security                                          5765
Climate change and energy                                  5675
Foreign affairs                                            5010
National Health Service                                    4970
Community and society                                      4334
Housing                                                    3996
Wildlife and animal welfare                                3944
Housing planning and building           

Administrative justice reform                             11717
Business and enterprise                                   11717
Adoption, fostering and surrogacy                         11717
Afghanistan                                               11717
Armed Forces Covenant                                     11717
Armed forces                                              11717
Armed forces and Ministry of Defence reform               11717
Government efficiency, transparency and accountability    11049
UK economy                                                10527
Trade and investment                                       7229
Public health                                              6156
National security                                          5765
Climate change and energy                                  5675
Foreign affairs                                            5010
National Health Service                                    4970
Community and society                   

Administrative justice reform                             11717
Adoption, fostering and surrogacy                         11717
Afghanistan                                               11717
Armed Forces Covenant                                     11717
Armed forces                                              11717
Armed forces and Ministry of Defence reform               11717
Armed forces support for activities in the UK             11717
Arts and culture                                          11717
Assessing environmental impact                            11717
Business and enterprise                                   11717
Government efficiency, transparency and accountability    11049
UK economy                                                10527
Trade and investment                                       7229
Public health                                              6156
National security                                          5765
Climate change and energy               

Administrative justice reform                             11717
Adoption, fostering and surrogacy                         11717
Afghanistan                                               11717
Armed Forces Covenant                                     11717
Armed forces                                              11717
Armed forces and Ministry of Defence reform               11717
Armed forces support for activities in the UK             11717
Arts and culture                                          11717
Assessing environmental impact                            11717
Asylum                                                    11717
Attorney General guidance to the legal profession         11717
Aviation                                                  11717
Business and enterprise                                   11717
Government efficiency, transparency and accountability    11049
UK economy                                                10527
Trade and investment                    

...

Lasting power of attorney, being in care and your financial affairs    11717
Land registration                                                      11717
Food and farming                                                       11717
Flooding and coastal change                                            11717
Fire prevention and rescue                                             11717
Financial services                                                     11717
Financial help if you have children                                    11717
Family justice system                                                  11717
Expenses and employee benefits                                         11717
European single market                                                 11717
European funds                                                         11717
European Union laws and regulation                                     11717
Equality, rights and citizenship                                       11717

Lasting power of attorney, being in care and your financial affairs    11717
Land registration                                                      11717
Food and farming                                                       11717
Flooding and coastal change                                            11717
Fire prevention and rescue                                             11717
Financial services                                                     11717
Financial help if you have children                                    11717
Family justice system                                                  11717
Expenses and employee benefits                                         11717
European single market                                                 11717
European funds                                                         11717
European Union laws and regulation                                     11717
Equality, rights and citizenship                                       11717

Lasting power of attorney, being in care and your financial affairs    11717
Land registration                                                      11717
Food and farming                                                       11717
Flooding and coastal change                                            11717
Fire prevention and rescue                                             11717
Financial services                                                     11717
Financial help if you have children                                    11717
Family justice system                                                  11717
Expenses and employee benefits                                         11717
European single market                                                 11717
European funds                                                         11717
European Union laws and regulation                                     11717
Equality, rights and citizenship                                       11717

Lasting power of attorney, being in care and your financial affairs    11717
Devolution                                                             11717
Forced marriage                                                        11717
Food and farming                                                       11717
Flooding and coastal change                                            11717
Fire prevention and rescue                                             11717
Financial services                                                     11717
Financial help if you have children                                    11717
Family justice system                                                  11717
Expenses and employee benefits                                         11717
European single market                                                 11717
European funds                                                         11717
European Union laws and regulation                                     11717

ValueError: low >= high

In [ ]:
balanced_df.to_csv(os.path.join(DATADIR, 'balanced_level2.csv.gz'), compression='gzip')

In [120]:
##Lazy copying here
labelled_level2 = balanced_df

### drop news

In [121]:
labelled_level2[(labelled_level2['document_type'] == 'world_news_story')].shape

(17367, 24)

In [122]:
labelled_level2[(labelled_level2['document_type'] == 'news_story')].shape

(309000, 24)

In [123]:
nonews = labelled_level2[(labelled_level2['document_type'] != 'news_story')]

nonews = nonews[nonews['document_type'] != 'world_news_story']

In [124]:
nonews.shape

(2134203, 24)

### Create dictionary mapping taxon codes to string labels

In [125]:
#Get the category numeric values (codes) and avoid zero-indexing
labels = nonews['level2taxon'].cat.codes + 1

#create dictionary of taxon category code to string label for use in model evaluation
labels_index = dict(zip((labels), nonews['level2taxon']))
labels_index

{1: 'Administrative justice reform',
 2: 'Adoption, fostering and surrogacy',
 3: 'Afghanistan',
 4: 'Armed Forces Covenant',
 5: 'Armed forces',
 6: 'Armed forces and Ministry of Defence reform',
 7: 'Armed forces support for activities in the UK',
 8: 'Arts and culture',
 9: 'Assessing environmental impact',
 10: 'Asylum',
 11: 'Attorney General guidance to the legal profession',
 12: 'Aviation',
 13: 'Benefits entitlement',
 14: 'Benefits for families',
 15: 'Biodiversity and ecosystems',
 16: 'Boating and inland waterways',
 17: 'Brexit',
 18: 'Brexit and the EU',
 19: 'British citizenship ',
 20: 'British nationals overseas',
 21: 'Business and enterprise',
 22: 'Business and the environment',
 23: 'Business tax',
 24: 'Byelaws',
 25: 'Carers and disability benefits',
 26: "Carers' health",
 27: 'Certificates, register offices, changes of name or gender',
 28: 'Charities, volunteering and honours',
 29: 'Child Benefit',
 30: 'Child maintenance reform',
 31: 'Childcare and early ye

In [126]:
print(len(labels_index))

210


### Create target/Y 

Note: when using the categorical_crossentropy loss, your targets should be in categorical format (e.g. if you have 10 classes, the target for each sample should be a 10-dimensional vector that is all-zeros expect for a 1 at the index corresponding to the class of the sample).

In multilabel learning, the joint set of binary classification tasks is expressed with label binary indicator array: each sample is one row of a 2d array of shape (n_samples, n_classes) with binary values:  
the one, i.e. the non zero elements, corresponds to the subset of labels.  
An array such as np.array([[1, 0, 0], [0, 1, 1], [0, 0, 0]]) represents label 0 in the first sample, labels 1 and 2 in the second sample, and no labels in the third sample.  
Producing multilabel data as a list of sets of labels may be more intuitive.

####  First reshape wide to get columns for each level2taxon and row number = number unique urls

In [127]:
#get a smaller copy of data for pivoting ease (think you can work from full data actually and other cols get droopedauto)

level2_reduced = nonews[['content_id', 'level2taxon', 'combined_text', 'title', 'description']].copy()

#how many level2taxons are there?
print('Number of unique level2taxons: {}'.format(level2_reduced.level2taxon.nunique()))

#count the number of taxons per content item into new column
level2_reduced['num_taxon_per_content'] = level2_reduced.groupby(["content_id"])['content_id'].transform("count")

#Add 1 because of zero-indexing to get 1-number of level2taxons as numerical targets
level2_reduced['level2taxon_code'] = level2_reduced.level2taxon.astype('category').cat.codes + 1

Number of unique level2taxons: 210


In [128]:
#how many level2taxons are there?
print('Number of unique level2taxons: {}'.format(labelled_level2.level2taxon.nunique()))

#count the number of taxons per content item into new column
labelled_level2['num_taxon_per_content'] = labelled_level2.groupby(["content_id"])['content_id'].transform("count")

#Add 1 because of zero-indexing to get 1-number of level2taxons as numerical targets
labelled_level2['level2taxon_code'] = labelled_level2.level2taxon.astype('category').cat.codes + 1

Number of unique level2taxons: 210


In [129]:
#reshape to wide per taxon and keep the combined text so indexing is consistent when splitting X from Y

multilabel = (level2_reduced.pivot_table(index=['content_id', 'combined_text', 'title', 'description' ], 
                  columns='level2taxon_code', 
                  values='num_taxon_per_content'))
print('level2reduced shape: {}'.format(level2_reduced.shape))
print('pivot table shape (no duplicates): {} '.format(multilabel.shape))
print(multilabel.head())

level2reduced shape: (2134203, 7)
pivot table shape (no duplicates): (92338, 210) 
level2taxon_code                                                                                                                                                                               1    \
content_id                           combined_text                                      title                                              description                                               
00029fa4-9b60-4285-898c-85ae8a6367f5 emma jones - small business crown representativ... emma jones - small business crown representative   as small business crown representative emma is ...  NaN   
00037b70-5b08-44c2-bf0a-fa8eb636a60b land remediation: bringing brownfield sites bac... land remediation: bringing brownfield sites bac... brochure showing uk expertise in land remediati...  NaN   
00037ee5-7b5e-452d-a233-af2c134f5bce steps 2 success:ni statistics from october 2014... steps 2 success:ni statistics from oc

In [130]:
#THIS IS WHY INDEXING IS NOT ZERO-BASED
#convert the number_of_taxons_per_content values to 1, meaning there was an entry for this taxon and this content_id, 0 otherwise
binary_multilabel = multilabel.notnull().astype('int')

In [131]:
#will convert columns to an array of shape
print('Shape of Y multilabel array before train/val/test split:{}'.format(binary_multilabel[list(binary_multilabel.columns)].values.shape))

Shape of Y multilabel array before train/val/test split:(92338, 210)


In [132]:
#convert columns to an array. Each row represents a content item, each column an individual taxon
binary_multilabel = binary_multilabel[list(binary_multilabel.columns)].values
print('Example row of multilabel array {}'.format(binary_multilabel[2]))

Example row of multilabel array [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [133]:
type(binary_multilabel)

numpy.ndarray

### Format metadata/X

In [134]:
# THESE DICTS NEED TO BE CREATED IN ALL DATA, OR DOC TYPES NOT PRESENT IN TRAINING MUST BE DELETED FROM NEW
#create dictionary of document type to content_id
doctype_dict = dict(zip(nonews['content_id'], nonews['document_type']))
frstpub_dict = dict(zip(nonews['content_id'], nonews['first_published_at']))
pubapp_dict = dict(zip(nonews['content_id'], nonews['publishing_app']))
puborg_dict = dict(zip(nonews['content_id'], nonews['primary_publishing_organisation']))
len(doctype_dict)

92338

In [135]:
#the pivot table has two indices
print(multilabel.index.names)
print(multilabel.columns)
#print(multilabel.head())

['content_id', 'combined_text', 'title', 'description']
Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            201, 202, 203, 204, 205, 206, 207, 208, 209, 210],
           dtype='int64', name='level2taxon_code', length=210)


In [136]:
#extract content_id index to df
meta1 = pd.DataFrame(multilabel.index.get_level_values('content_id'))

In [137]:
#apply meta data to content
meta1['doctype'] = meta1['content_id'].map(doctype_dict).copy()
meta1['first_published_at'] = meta1['content_id'].map(frstpub_dict).copy()
meta1['pub_app'] = meta1['content_id'].map(pubapp_dict).copy()
meta1['prim_pub_org'] = meta1['content_id'].map(puborg_dict).copy()

meta1 = meta1.replace(np.nan, '', regex=True) #conver nans to empty strings for labelencoder types
meta1.head()

,content_id,doctype,first_published_at,pub_app,prim_pub_org
0,00029fa4-9b60-4285-898c-85ae8a6367f5,document_collection,2017-01-11T10:18:00.000+00:00,whitehall,{'title': 'Crown Commercial Service'}
1,00037b70-5b08-44c2-bf0a-fa8eb636a60b,promotional,2015-05-11T10:41:00.000+00:00,whitehall,{'title': 'Department for International Trade'}
2,00037ee5-7b5e-452d-a233-af2c134f5bce,official_statistics,2016-11-24T09:30:13.000+00:00,whitehall,{'title': 'Department for Communities (Norther...
3,0004c63d-ae16-432a-bb35-c0f949b1e27c,official_statistics,2016-10-20T08:30:09.000+00:00,whitehall,{'title': 'Welsh Government'}
4,0005ac76-50fe-42f1-8168-8b6fc046e40f,guidance,2017-08-02T17:29:00.000+00:00,whitehall,{'title': 'Department for Communities and Loca...


In [138]:
#encode strings as integers
#doctype
doctype_encoder = LabelEncoder()
meta1['doctype_cat'] = doctype_encoder.fit_transform(meta1['doctype']) #fit the label encoder on all data then transform on individual data sets

# print(len(meta1['doctype_cat'].unique()))
# print(meta1['doctype_cat'].unique())

# primary publishing org
pub_org_encoder = LabelEncoder()
meta1['prim_pub_org_cat'] = doctype_encoder.fit_transform(meta1['prim_pub_org']) 

# print(len(meta1['prim_pub_org_cat'].unique()))
# print(meta1['prim_pub_org_cat'].unique())

#publishing application

pub_app_encoder = LabelEncoder()
meta1['pub_app_cat'] = doctype_encoder.fit_transform(meta1['pub_app'])

# print(len(meta1['pub_app_cat'].unique()))
# print(meta1['pub_app_cat'].unique())

In [139]:
#one hot encode integers
encoded_doctype = to_categorical(meta1['doctype_cat'])
print(encoded_doctype.shape)

encoded_prim_pub_org = to_categorical(meta1['prim_pub_org_cat'])
print(encoded_prim_pub_org.shape)

encoded_pub_app = to_categorical(meta1['pub_app_cat'])
print(encoded_pub_app.shape)
type(encoded_pub_app)

(92338, 56)
(92338, 366)
(92338, 9)


numpy.ndarray

In [140]:
meta1['first_published_at'] = pd.to_datetime(meta1['first_published_at']) 

# print(meta['first_published_at'][0].Timestamp())
# for index, row in meta.iterrows():
#     row['first_published_at_ts'] = row['first_published_at'].timestamp()
#     print(row['first_published_at_ts'])

print(meta1['first_published_at'].shape)

first_published = np.array(meta1['first_published_at']).reshape(meta1['first_published_at'].shape[0], 1).copy()

print(first_published.dtype)
print(np.argmax(first_published.argmax))
#scale first published at to 0 and 1
print(first_published.shape)
type(first_published)

(92338,)
datetime64[ns]
0
(92338, 1)


numpy.ndarray

In [141]:

meta = np.concatenate((encoded_doctype, 
                           encoded_prim_pub_org, 
                           encoded_pub_app), #error when adding date...
                          axis=1)

print(meta.shape)
nb_metavars = meta.shape[1]
print(nb_metavars)
print(meta.shape)

(92338, 431)
431
(92338, 431)


### Create combined_text data/X

format our text samples and labels into tensors that can be fed into a neural network. To do this, we will rely on Keras utilities keras.preprocessing.text.Tokenizer and keras.preprocessing.sequence.pad_sequences.

In [142]:
#the pivot table has two indices
multilabel.index.names

FrozenList(['content_id', 'combined_text', 'title', 'description'])

In [143]:
#extract combined text index to array
texts = multilabel.index.get_level_values('combined_text')
texts.shape

(92338,)

### Preprocess title data

In [144]:
titles = multilabel.index.get_level_values('title')
titles.shape

(92338,)

### Preprocess description data

In [145]:
descs = multilabel.index.get_level_values('description')
descs.shape

(92338,)

### Tokenize combined text

Tokenizer = Class for vectorizing texts, or/and turning texts into sequences (=list of word indexes, where the word of rank i in the dataset (starting at 1) has index i)

In [ ]:
# Bag of words method
tokenizer = Tokenizer(num_words=NUM_WORDS) #num_words: None or int. Maximum number of words to work with 
# (if set, tokenization will be restricted to the top num_words most common words in the dataset).

# apply tokenizer to our text data
tokenizer.fit_on_texts(texts)

# list of word indexes, where the word of rank i in the dataset (starting at 1) has index i
sequences = tokenizer.texts_to_sequences(texts) #yield one sequence per input text

# dictionary mapping words (str) to their rank/index (int).
word_index = tokenizer.word_index  # Only set after fit_on_texts was called.
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen= MAX_SEQUENCE_LENGTH) #MAX_SEQUENCE_LENGTH

In [ ]:
print('Shape of label tensor:', binary_multilabel.shape)
print('Shape of data tensor:', data.shape)

### Tokenize title text

In [ ]:
# Bag of words method
tokenizer_tit = Tokenizer(num_words=10000) #num_words: None or int. Maximum number of words to work with 
# (if set, tokenization will be restricted to the top num_words most common words in the dataset).

# apply tokenizer to our text data
tokenizer_tit.fit_on_texts(titles)

# list of word indexes, where the word of rank i in the dataset (starting at 1) has index i
sequences_tit = tokenizer_tit.texts_to_sequences(titles) #yield one sequence per input text

In [ ]:
onehot_tit = tokenizer_tit.sequences_to_matrix(sequences_tit) #yield one sequence per input text

# dictionary mapping words (str) to their rank/index (int).
word_index_tit = tokenizer_tit.word_index  # Only set after fit_on_texts was called.

#NOTE THERE ARE LOTS OF NUMBERS IN HERE THAT SHOULD BE STRIPPED?

# print('Found %s unique tokens.' % len(word_index_tit))

# title_data = pad_sequences(sequences_tit, maxlen= 50) #mean/median=8 max = 47

In [ ]:
onehot_tit.shape

In [ ]:

sorted(tokenizer_tit.word_counts.items(), key=lambda x:x[1], reverse=True)
#for key, value in sorted(mydict.iteritems(), key=lambda (k,v): (v,k)):
    #print "%s: %s" % (key, value)

### Tokenize description text

In [ ]:
# Bag of words method
tokenizer_desc = Tokenizer(num_words=10000) #num_words: None or int. Maximum number of words to work with 
# (if set, tokenization will be restricted to the top num_words most common words in the dataset).

# apply tokenizer to our text data
tokenizer_desc.fit_on_texts(descs)

# list of word indexes, where the word of rank i in the dataset (starting at 1) has index i
sequences_desc = tokenizer_desc.texts_to_sequences(descs) #yield one sequence per input text
onehot_desc = tokenizer_desc.sequences_to_matrix(sequences_desc) #yield one sequence per input text

# dictionary mapping words (str) to their rank/index (int).
word_index_desc = tokenizer_desc.word_index 

In [ ]:
onehot_desc.shape

### Data split
- Training data = 80%
- Development data = 10%
- Test data = 10%

In [ ]:
# shuffle data and standardise indices
indices = np.arange(data.shape[0])
print(indices)
np.random.seed(0)
np.random.shuffle(indices)
print(indices)

In [ ]:
data = data[indices]
metadata = meta[indices]
title_data = onehot_tit[indices]

In [ ]:
desc_data = onehot_desc[indices]
timedata = first_published[indices]


labels = binary_multilabel[indices]

In [ ]:
nb_test_samples = int(0.1 * data.shape[0]) #validation split
print('nb_test samples:', nb_test_samples)

nb_dev_samples = int(0.2 * data.shape[0]) #validation split
print('nb_dev samples:', nb_dev_samples)

nb_training_samples = int(0.8 * data.shape[0]) #validation split
print('nb_training samples:', nb_training_samples)

In [ ]:
x_train = data[:-nb_dev_samples]
print('Shape of x_train:', x_train.shape)

metax_train = metadata[:-nb_dev_samples]
print('Shape of metax_train:', metax_train.shape)

titlex_train = title_data[:-nb_dev_samples]
print('Shape of titlex_train:', titlex_train.shape)

descx_train = desc_data[:-nb_dev_samples]
print('Shape of descx_train:', descx_train.shape)

datex_train = timedata[:-nb_dev_samples]
print('Shape of datex_train:', datex_train.shape)

y_train = labels[:-nb_dev_samples]
print('Shape of y_train:', y_train.shape)

In [ ]:
x_dev = data[-nb_dev_samples:-nb_test_samples]
print('Shape of x_dev:', x_dev.shape)

metax_dev = metadata[-nb_dev_samples:-nb_test_samples]
print('Shape of metax_dev:', metax_dev.shape)

titlex_dev = title_data[-nb_dev_samples:-nb_test_samples]
print('Shape of titlex_dev:', titlex_dev.shape)

descx_dev = desc_data[-nb_dev_samples:-nb_test_samples]
print('Shape of descx_dev:', descx_dev.shape)

datex_dev = timedata[-nb_dev_samples:-nb_test_samples]
print('Shape of metax_dev:', datex_dev.shape)

y_dev = labels[-nb_dev_samples:-nb_test_samples]
print('Shape of y_dev:', y_dev.shape)

In [ ]:
x_test = data[-nb_test_samples:]
print('Shape of x_test:', x_test.shape)

metax_test = metadata[-nb_test_samples:]
print('Shape of metax_test:', metax_test.shape)

titlex_test = title_data[-nb_test_samples:]
print('Shape of titlex_test:', titlex_test.shape)

descx_test = desc_data[-nb_test_samples:]
print('Shape of descx_test:', descx_test.shape)

datex_test = timedata[-nb_test_samples:]
print('Shape of datex_test:', datex_test.shape)

y_test = labels[-nb_test_samples:]
print('Shape of y_test:', y_test.shape)

### preparing the Embedding layer

NB stopwords haven't been removed yet...

In [ ]:
embedding_layer = Embedding(len(word_index) + 1, 
                            EMBEDDING_DIM, 
                            input_length=MAX_SEQUENCE_LENGTH)

An Embedding layer should be fed sequences of integers, i.e. a 2D input of shape (samples, indices). These input sequences should be padded so that they all have the same length in a batch of input data (although an Embedding layer is capable of processing sequence of heterogenous length, if you don't pass an explicit input_length argument to the layer).

All that the Embedding layer does is to map the integer inputs to the vectors found at the corresponding index in the embedding matrix, i.e. the sequence [1, 2] would be converted to [embeddings[1], embeddings[2]]. This means that the output of the Embedding layer will be a 3D tensor of shape (samples, sequence_length, embedding_dim).

### Estimate class weights for unbalanced datasets.
paramter to model.fit = __class_weight__: Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). This can be useful to tell the model to "pay more attention" to samples from an under-represented class.

Implement class_weight from sklearn:

- Import the module 

`from sklearn.utils import class_weight`
- calculate the class weight, If ‘balanced’, class weights will be given by n_samples / (n_classes * np.bincount(y)):

`class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)`

- change it to a dict in order to work with Keras.

`class_weight_dict = dict(enumerate(class_weight))`

- Add to model fitting

`model.fit(X_train, y_train, class_weight=class_weight)`

In [ ]:
# class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
# class_weight_dict = dict(enumerate(class_weight))

### Custom loss function

In [ ]:
class WeightedBinaryCrossEntropy(object):

    def __init__(self, pos_ratio):
        neg_ratio = 1. - pos_ratio
        #self.pos_ratio = tf.constant(pos_ratio, tf.float32)
        self.pos_ratio = pos_ratio
        #self.weights = tf.constant(neg_ratio / pos_ratio, tf.float32)
        self.weights = neg_ratio / pos_ratio
        self.__name__ = "weighted_binary_crossentropy({0})".format(pos_ratio)

    def __call__(self, y_true, y_pred):
        return self.weighted_binary_crossentropy(y_true, y_pred)

    def weighted_binary_crossentropy(self, y_true, y_pred):
            # Transform to logits
            epsilon = tf.convert_to_tensor(K.common._EPSILON, y_pred.dtype.base_dtype)
            y_pred = tf.clip_by_value(y_pred, epsilon, 1 - epsilon)
            y_pred = tf.log(y_pred / (1 - y_pred))

            cost = tf.nn.weighted_cross_entropy_with_logits(y_true, y_pred, self.weights)
            return K.mean(cost * self.pos_ratio, axis=-1)
    
y_true_arr = np.array([0,1,0,1], dtype="float32")
y_pred_arr = np.array([0,0,1,1], dtype="float32")
y_true = tf.constant(y_true_arr)
y_pred = tf.constant(y_pred_arr)

with tf.Session().as_default(): 
    print(WeightedBinaryCrossEntropy(0.5)(y_true, y_pred).eval())
    print(binary_crossentropy(y_true, y_pred).eval())


### difficulty getting global precision/recall metrics . CAUTION interpreting monitoring metrics
fcholltet: "Basically these are all global metrics that were approximated
batch-wise, which is more misleading than helpful. This was mentioned in
the docs but it's much cleaner to remove them altogether. It was a mistake
to merge them in the first place."

In [ ]:
def f1(y_true, y_pred):
    """Use Recall  and precision metrics to calculate harmonic mean (F1 score).

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1 = 2*((precision*recall)/(precision+recall))
    
    return f1

## Training a 1D convnet

### 1. Create model

In [ ]:
NB_CLASSES = y_train.shape[1]
NB_METAVARS = metax_train.shape[1]



sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32', name='wordindex') #MAX_SEQUENCE_LENGTH
embedded_sequences = embedding_layer(sequence_input)
x = Dropout(0.2, name = 'dropout_embedded')(embedded_sequences)

x = Conv1D(128, 5, activation='relu', name = 'conv0')(x)

x = MaxPooling1D(5, name = 'max_pool0')(x)

x = Dropout(0.5, name = 'dropout0')(x)

x = Conv1D(128, 5, activation='relu', name = 'conv1')(x)

x = MaxPooling1D(5 , name = 'max_pool1')(x)

x = Conv1D(128, 5, activation='relu', name = 'conv2')(x)

x = MaxPooling1D(35, name = 'global_max_pool')(x)  # global max pooling

x = Flatten()(x) #reduce dimensions from 3 to 2; convert to vector + FULLYCONNECTED

meta_input = Input(shape=(NB_METAVARS,), name='meta')
meta_hidden = Dense(128, activation='relu', name = 'hidden_meta')(meta_input)
meta_hidden = Dropout(0.2, name = 'dropout_meta')(meta_hidden)


title_input = Input(shape=(titlex_train.shape[1],), name='titles')
title_hidden = Dense(128, activation='relu', name = 'hidden_title')(title_input)
title_hidden = Dropout(0.2, name = 'dropout_title')(title_hidden)

desc_input = Input(shape=(descx_train.shape[1],), name='descs')
desc_hidden = Dense(128, activation='relu', name = 'hidden_desc')(desc_input)
desc_hidden = Dropout(0.2, name = 'dropout_desc')(desc_hidden)

concatenated = concatenate([meta_hidden, title_hidden, desc_hidden, x])

x = Dense(400, activation='relu', name = 'fully_connected0')(concatenated)

x = Dropout(0.2, name = 'dropout1')(x)

x = Dense(NB_CLASSES, activation='sigmoid', name = 'fully_connected1')(x)

# # The Model class turns an input tensor and output tensor into a model
# This creates Keras model instance, will use this instance to train/test the model.
model = Model(inputs=[meta_input, title_input, desc_input, sequence_input], outputs=x)

### 2. Compile model

In [ ]:
model.compile(loss=WeightedBinaryCrossEntropy(POS_RATIO),
              optimizer='rmsprop',
              metrics=['binary_accuracy', f1])

Metric values are recorded at the end of each epoch on the training dataset. If a validation dataset is also provided, then the metric recorded is also calculated for the validation dataset.

All metrics are reported in verbose output and in the history object returned from calling the fit() function. In both cases, the name of the metric function is used as the key for the metric values. In the case of metrics for the validation dataset, the “val_” prefix is added to the key.

You have now built a function to describe your model. To train and test this model, there are four steps in Keras:
1. Create the model by calling the function above
2. Compile the model by calling `model.compile(optimizer = "...", loss = "...", metrics = ["accuracy"])`
3. Train the model on train data by calling `model.fit(x = ..., y = ..., epochs = ..., batch_size = ...)`
4. Test the model on test data by calling `model.evaluate(x = ..., y = ...)`

If you want to know more about `model.compile()`, `model.fit()`, `model.evaluate()` and their arguments, refer to the official [Keras documentation](https://keras.io/models/model/).


In [ ]:
model.summary()

### Tensorboard callbacks /metrics /monitor training

<span style="color:red"> **Size of these files is killing storage during training. Is it histograms?**</span>

In [ ]:
tb = TensorBoard(log_dir='./learn_embedding_logs', histogram_freq=1, write_graph=True, write_images=False)

In [ ]:
CHECKPOINT_PATH = os.path.join(DATADIR, 'model_checkpoint.hdf5')

cp = ModelCheckpoint(
                     filepath = CHECKPOINT_PATH, 
                     monitor='val_loss', 
                     verbose=0, 
                     save_best_only=False, 
                     save_weights_only=False, 
                     mode='auto', 
                     period=1
                    )

In [ ]:
# class Metrics(Callback):
#     def on_train_begin(self, logs={}):
#         self.val_f1s = []
#         self.val_recalls = []
#         self.val_precisions = []
 
#     def on_epoch_end(self, epoch, logs={}):
#         val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
#         val_targ = self.model.validation_data[1]
        
#         self.val_f1s.append(f1_score(val_targ, val_predict, average='micro'))
#         self.val_recalls.append(recall_score(val_targ, val_predict))
#         self.val_precisions.append(precision_score(val_targ, val_predict))
#         print("- val_f1: %f — val_precision: %f — val_recall %f" 
#                 %(f1_score(val_targ, val_predict, average='micro'), 
#                   precision_score(val_targ, val_predict),
#                    recall_score(val_targ, val_predict)))
#         return
 
# metrics = Metrics()

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
#model.fit(x, y, validation_split=0.2, callbacks=[early_stopping])

### 3. Train model

In [ ]:
# metrics callback causes: CCCCCCR55555555511155
# So disable for now

history = model.fit(
    {'meta': metax_train, 'titles': titlex_train, 'descs': descx_train, 'wordindex': x_train},
    y_train, 
    validation_data=([metax_dev, titlex_dev, descx_dev, x_dev], y_dev), 
    epochs=10, batch_size=128, callbacks=[early_stopping]
)

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, 8)

plt.plot(epochs, loss_values, 'bo', label='Training loss')           
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')      
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()    

f1_values = history_dict['f1']
val_f1_values = history_dict['val_f1']

plt.plot(epochs, f1_values, 'bo', label='Training f1')
plt.plot(epochs, val_f1_values, 'b', label='Validation f1')
plt.title('Training and validation batch-level f1-micro')
plt.xlabel('Epochs')
plt.ylabel('F1-micro')
plt.legend()

plt.show()

### Evaluate model

#### Training metrics

In [ ]:
y_prob = model.predict([metax_train, titlex_train, descx_train, x_train])

In [ ]:
y_prob.shape

In [ ]:
y_pred = y_prob.copy()
y_pred[y_pred>=P_THRESHOLD] = 1
y_pred[y_pred<P_THRESHOLD] = 0

In [ ]:
f1_score(y_train, y_pred, average='micro')

In [ ]:
#average= None, the scores for each class are returned.
precision_recall_fscore_support(y_train, y_pred, average=None, sample_weight=None)

In [ ]:
a = precision_recall_fscore_support(y_train, y_pred, average=None, sample_weight=None)

In [ ]:
pd.DataFrame(list(a))

In [ ]:
f1_byclass = pd.DataFrame((a)[2], columns=['f1'])


In [ ]:
support_byclass = pd.DataFrame((a)[3], columns=['support'])

f1_byclass = pd.merge(
    left=f1_byclass, 
    right=support_byclass, 
    left_index=True,
    right_index=True,
    how='outer', 
    validate='one_to_one'
)

f1_byclass['index_col'] = f1_byclass.index

In [ ]:
f1_byclass['level2taxon'] = f1_byclass['index_col'].map(labels_index).copy()

In [ ]:
print("At p_threshold of {}, there were {} out of {} ({})% taxons with auto-tagged content in the training data"
      .format(P_THRESHOLD, 
              f1_byclass.loc[f1_byclass['f1'] > 0].shape[0], 
              y_pred.shape[1], 
              (f1_byclass.loc[f1_byclass['f1'] > 0].shape[0]/y_pred.shape[1])*100 ))

In [ ]:
no_auto_content = f1_byclass.loc[f1_byclass['f1'] == 0]
no_auto_content = no_auto_content.set_index('level2taxon')

In [ ]:
no_auto_content['support'].sort_values().plot( kind = 'barh', figsize=(20, 20))

In [ ]:
classes_predictedto = f1_byclass.loc[f1_byclass['f1'] > 0]
classes_predictedto = classes_predictedto.set_index('level2taxon') 

In [ ]:
classes_predictedto.plot.scatter(x='support', y='f1', figsize=(20, 10), xticks=np.arange(0, 9700, 100))

In [ ]:
classes_predictedto['f1'].sort_values().plot( kind = 'barh', figsize=(20, 20))

In [ ]:
#Calculate globally by counting the total true positives, false negatives and false positives.
precision_recall_fscore_support(y_train, y_pred, average='micro', sample_weight=None) 

In [ ]:
#Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account
precision_recall_fscore_support(y_train, y_pred, average='macro', sample_weight=None)

In [ ]:
#Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account
precision_recall_fscore_support(y_train, y_pred, average='weighted', sample_weight=None)

#### Development set metrics

In [ ]:
y_pred_dev = model.predict([metax_dev, titlex_dev, descx_dev, x_dev])

In [ ]:
y_pred_dev[y_pred_dev>=P_THRESHOLD] = 1
y_pred_dev[y_pred_dev<P_THRESHOLD] = 0

In [ ]:
#average= None, the scores for each class are returned.
precision_recall_fscore_support(y_dev, y_pred_dev, average=None, sample_weight=None)

In [ ]:
#Calculate globally by counting the total true positives, false negatives and false positives.
precision_recall_fscore_support(y_dev, y_pred_dev, average='micro', sample_weight=None) 

In [ ]:
#Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account
precision_recall_fscore_support(y_dev, y_pred_dev, average='macro', sample_weight=None)

In [ ]:
#Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account
precision_recall_fscore_support(y_dev, y_pred_dev, average='weighted', sample_weight=None)

## Tag unlabelled content

In [ ]:
def get_predictions(new_texts, df, level1taxon=False):
    #process data for model input
    
    new_sequences = tokenizer.texts_to_sequences(new_texts) #yield one sequence per input text

    new_word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(new_word_index))

    x_new = pad_sequences(new_sequences, maxlen= MAX_SEQUENCE_LENGTH) #MAX_SEQUENCE_LENGTH
    
    print('Shape of untagged tensor:', x_new.shape)
    
    #predict tag for untagged data
    y_pred_new = model.predict(x_new)
    
    #get model output into pandas & get a column to track index for later merge
    y_pred_new = pd.DataFrame(y_pred_new)
    y_pred_new['index_col'] = y_pred_new.index
    
    #Make long by taxon so easier to filter rows and examine effect of p_threshold
    y_pred_new = pd.melt(y_pred_new, id_vars=['index_col'],
                             var_name='level2taxon_code', value_name='probability')
    
    #get taxon names
    y_pred_new['level2taxon'] = y_pred_new['level2taxon_code'].map(labels_index)
    
    if level1taxon==False:
        #get the info about the content
        new_info = df[[ 'base_path', 'content_id', 'title', 'description', 
                   'document_type', 'publishing_app', 'locale']]
    else:
        new_info = df[[ 'base_path', 'content_id', 'title', 'description', 
                   'document_type', 'publishing_app', 'locale', 'level1taxon']]
    
    
    #merge content info with taxon prediction
    pred_new = pd.merge(
    left=new_info, 
    right=y_pred_new, 
    left_index=True,
    right_on='index_col',
    how='outer'
    )
    
    #drop the cols needed for mergingin and naming
    pred_new.drop(['index_col'], axis=1, inplace = True)
    
    #keep only rows where prob of taxon > 0.5
    
    
    return pred_new #.loc[pred_new['probability'] > P_THRESHOLD] #only return rows/samples where probability is hihger than threshold
    

### Untagged

In [ ]:
#read in untagged content
untagged_raw = pd.read_csv(os.path.join(DATADIR, 'untagged_content.csv.gz'), dtype=object, compression='gzip')

In [ ]:
untagged_raw.head()

In [ ]:
new_texts = untagged_raw['combined_text']

In [ ]:
pred_untagged = get_predictions(new_texts, untagged_raw)

In [ ]:
#data is long by taxon
print('Number of unique content items: {}'.format(pred_untagged.content_id.nunique()))
print('Number of content items tagged to taxons with more than p_threshold: {}'.format(pred_untagged.shape))

In [ ]:
pred_untagged.loc[(pred_untagged['probability'] > 0.65) & (pred_untagged['probability'] < 0.85)].sort_values(by='probability', ascending=False)

In [ ]:
#write to csv
pred_untagged.to_csv(os.path.join(DATADIR, 'predictions_for_untagged_data_trainingdatatok.csv'), index=False)

In [ ]:
# apply tokenizer to our text data
tokenizer.fit_on_texts(new_texts)

pred_untagged_refit_tok = get_predictions(new_texts, untagged_raw)


In [ ]:
#write to csv
pred_untagged_refit_tok.to_csv(os.path.join(DATADIR, 'predictions_for_untagged_data_refittok.csv'), index=False)

### New data (untagged + old taxons)

old_taxons data has no combined text. This needs fixing in the data pipeline before being able to use these data for predictions

In [ ]:
#read in untagged content
new_raw = pd.read_csv(os.path.join(DATADIR, 'new_content.csv'), dtype=object)

In [ ]:
new_raw.shape

In [ ]:
type(new_raw['combined_text'][0])

In [ ]:
new_raw['combined_text'][0]

In [ ]:
len(new_raw[new_raw['combined_text'].isna()])

In [ ]:
(new_raw.loc[(new_raw['combined_text'].isna()) & (new_raw['untagged_type'] == 'old_taxons')]).shape

In [ ]:
new_raw[new_raw.untagged_type == 'old_taxons']

In [ ]:
#make a copy so you can edit data without needed to read in each time
new_df = new_raw.copy(deep=True)

In [ ]:
pred_new = get_predictions(new_df )

In [ ]:
#keep only rows where prob of taxon > 0.5
pred_new = pred_new.loc[pred_new['probability'] > 0.5]

In [ ]:
#write to csv
pred_new.to_csv(os.path.join(DATADIR, 'predictions_for_new_data.csv'), index=False)

### Labelled at level1only

In [ ]:
labelled_level1 = pd.read_csv(os.path.join(DATADIR, 'labelled_level1.csv'), dtype=object)

In [ ]:
level1_texts = labelled_level1['combined_text']

In [ ]:
#reset tokenizer to training data texts
tokenizer.fit_on_texts(texts)

In [ ]:
pred_labelled_level1 = get_predictions(level1_texts, labelled_level1, level1taxon=True)

In [ ]:
pred_labelled_level1.sort_values(by='probability', ascending=False)

In [ ]:
#write to csv
pred_labelled_level1.to_csv(os.path.join(DATADIR, 'predictions_for_level1only.csv'), index=False)

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='cnn.png', show_shapes=True)